In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Define the preprocessing function
def preprocess(image, label):
    # Apply data augmentation
    image = tf.image.random_flip_left_right(image)
    image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    # image = tf.image.random_zoom(image, [0.9, 1.1], [0.9, 1.1])
    image = tf.image.random_crop(image, size=[img_height, img_width, 3])
    # Normalize the images to the range the pre-trained model expects
    image = tf.keras.applications.efficientnet.preprocess_input(image)
    return image, label

# Create the base model from the pre-trained EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)  # Set the number of classes here

inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)


# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 30  # Keeping a higher number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved as 'model.tflite'")print(f'Test accuracy: {accuracy}')

Found 800 files belonging to 20 classes.
Found 200 files belonging to 20 classes.
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 418ms/step - accuracy: 0.3182 - loss: 2.3578 - val_accuracy: 0.7650 - val_loss: 0.7556 - learning_rate: 0.0010
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 320ms/step - accuracy: 0.9063 - loss: 0.3564 - val_accuracy: 0.8900 - val_loss: 0.4605 - learning_rate: 0.0010
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 329ms/step - accuracy: 0.9510 - loss: 0.1837 - val_accuracy: 0.9100 - val_loss: 0.3112 - learning_rate: 0.0010
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 335ms/step - accuracy: 0.9900 - loss: 0.0619 - val_accuracy: 0.9350 - val_loss: 0.2664 - learning_rate: 0.0010
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 328ms/step - accuracy: 0.9861 - loss: 0.0493 - val_accuracy: 0.9450 - val_loss: 0.2264 - learning_rate: 0.0010
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 338ms/step - accuracy: 0.9906 - loss: 0.0353 - val_accuracy: 0.9300 - val_loss: 0.2895 - learning_rate: 0.0010
Epoch

In [7]:
# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved as 'model.tflite'")

# Convert model to .h5
model.save('model4.h5')

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('model3.h5')

# Define function to preprocess an image file for prediction
def preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Example usage: Predicting on a single image
image_path = 'dataset/uji/rawon2.jpg'
preprocessed_img = preprocess_image(image_path)
predictions = model.predict(preprocessed_img)
predicted_class = np.argmax(predictions[0])
confidence = predictions[0][predicted_class]

# Retrieve class labels
class_names_file = 'labels.txt'
with open(class_names_file, 'r') as f:
    class_names = f.read().splitlines()

# Print results
print(f'Predicted class: {class_names[predicted_class]}')
print(f'Confidence: {confidence:.2f}')

1/1 [==============================] - 1s 836ms/step
Predicted class: Rawon: 400-500 kalori
Confidence: 1.00
